In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
nltk.download('punkt')   #For tokenization
nltk.download('stopwords')  #For removing the commonly used term(like 'and','is','was','they',etc.)

[nltk_data] Downloading package punkt to C:\Users\Aaditya
[nltk_data]     Mehta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Aaditya
[nltk_data]     Mehta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
data=pd.read_csv("Flight_Booking_System.flights.csv")

In [4]:
def preprocess_text(text):    #This is the data pre-processing 
    stop_words = set(stopwords.words('english'))    #removing unnecessary words
    tokens = word_tokenize(text)    #tokens contains the tokenization of the sentence provided 
    sample=tokens     #Taken a clone of original sentence, so that we can add the stopwords we require
    # Remove stopwords and non-alphabetic tokens
    tokens = [word.lower() for word in tokens if word.isalpha() and word not in stop_words]  #only the words which are alphabets and not in stop_words(The common words) will remain in tokens
   #Adding the stopwords that are required
    if "from" in sample:
        tokens.append('from')
    if "to" in sample:
        tokens.append('to')
    if "where" in sample:
        tokens.append('where')
    if "which" in sample:
        tokens.append('which')
    if "how" in sample:
        tokens.append('how')
    if "time" in sample:
        tokens.append("time")
    if "long" in sample:
        tokens.append("long")
    return tokens



In [5]:
def check_flight_id(flight_id, data):    #This will check if the entered flight_id exists in 'data' dataframe
    return flight_id in data['_id'].values    #This will return a boolean value

In [6]:
def handle_specific_query(flight_id, query, data):
    tokens = preprocess_text(query)   #query is the input statement given by the user
    flight_info = data[data['_id'] == flight_id].iloc[0]   #This will give the row where the flight_id equals the input flight_id. iloc[0] is used to access the first row of the dataframe, wherein you get the column titles in the sub-dataframe(which has the flight_id row).
    # Conditional statements for each question type
    if any(word in tokens for word in ['airline', 'which airline','airlines','which airlines']): #if the input sentence contains any of the following words, then if statement holds true. This is true for all if statements below.
        return f"The airline is {flight_info['airline']}."
    
    elif any(word in tokens for word in ['flight no', 'flight number', 'number']):
        return f"The flight number is {flight_info['flightNumber']}."
    
    elif any(word in tokens for word in ['source','depart','departing','from','coming','heading from']):
        return f"The flight origin is {flight_info['origin']}."
    
    elif any(word in tokens for word in ['destination','landing','arrive','land','arrival','going','to','arriving']):
        return f"The flight destination is {flight_info['destination']}."

    
    elif any(word in tokens for word in ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']):
        days_of_week = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
        found_day = None
        for day in days_of_week:
            if day in tokens:  
                found_day = day
                break
        
        if found_day is not None:
            if flight_info[found_day.capitalize()] == 'Y':
                return f"Yes, this flight flies on {found_day.capitalize()}."
            else:
                return f"No, this flight doesn't fly on {found_day.capitalize()}."
        else:
            return "Day of the week not recognized in query."
    
    
    elif any(word in tokens for word in ['weekend','weekends']):
        if flight_info['Saturday'] == 'Y' or flight_info['Sunday'] == 'Y':
            return "Yes, this flight flies on the weekends."
        else:
            return "No, this flight doesn't fly on the weekends."
        
    elif any(word in tokens for word in ['weekday','weekdays']):
        if flight_info['Monday']=='Y' or flight_info['Tuesday']=='Y' or flight_info['Wednesday']=='Y' or flight_info['Thursday']=='Y' or flight_info['Friday']=='N':
            return "Yes, this flight flies on the weekdays."
        else:
            return "No, this flight doesn't fly on the weekdays."

    
    elif any(word in tokens for word in ['scheduledDepartureTime','departure']):
        return f"The scheduled departure time is {flight_info['scheduledDepartureTime']}."
    
    elif any(word in tokens for word in ['scheduledArrivalTime','arrival']):
        return f"The scheduled arrival time is {flight_info['scheduledArrivalTime']}."
    
    elif any(word in tokens for word in ['flighttime', 'time', 'long', 'duration']):
        if 'long' in tokens or 'time' or 'duration' in tokens:
            return f"The flight time in hours is {flight_info['flightTime']}."
        else:
            return "You can ask about the airline, flight number, origin, destination, operation days, scheduled times, flight time, rate, or seat availability."

    
    elif any(word in tokens for word in ['rate', 'price']):
        return f"The rate/price of the flight is {flight_info['rate']}."
    
    elif any(word in tokens for word in ['seats', 'available','avail']):
        return f"There are {flight_info['seatAvail']} seats available."
    
    else:
        return "You can ask about the airline, flight number, origin, destination, operation days, scheduled times, flight time, rate, or seat availability."



In [7]:
# Main interaction loop
flight_id = input("Please enter the flight ID you're inteqrested in: ")
if check_flight_id(flight_id, data):
    print("Flight ID recognized. You can now ask specific questions about this flight.")
    while True:
        user_query = input("Ask me about the source, destination, flight minutes, rate, or days of operation (or type 'quit' to exit): ")
        if user_query.lower() == 'quit':
            break
        response = handle_specific_query(flight_id, user_query, data)  # Pass the data argument here
        print(response)
else:
    print("Flight ID not found. Please try again with a valid flight ID.")

Please enter the flight ID you're inteqrested in: 65e3679ccb66225971331578
Flight ID recognized. You can now ask specific questions about this flight.
Ask me about the source, destination, flight minutes, rate, or days of operation (or type 'quit' to exit): What is the price of this flight?
The rate/price of the flight is 3740.0.
Ask me about the source, destination, flight minutes, rate, or days of operation (or type 'quit' to exit): What is the departure time ?
The scheduled departure time is 00:10.
Ask me about the source, destination, flight minutes, rate, or days of operation (or type 'quit' to exit): What is the arrival time?
The flight destination is Bengaluru.
Ask me about the source, destination, flight minutes, rate, or days of operation (or type 'quit' to exit): From where will this flight depart?
The flight origin is Mumbai.
Ask me about the source, destination, flight minutes, rate, or days of operation (or type 'quit' to exit): Where will this flight land?
The flight dest